<a href="https://colab.research.google.com/github/records-sa/Python/blob/main/Colab_2%EC%A3%BC%EC%B0%A8(yahoo_query_%EC%8B%A4%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install urllib3==1.26.14 requests==2.25.1 yahooquery==2.3.0

In [54]:
from yahooquery import Ticker
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [38]:
def add_company(code):

  company = Ticker(code)

  modules = 'summaryDetail assetProfile price financialData defaultKeyStatistics'
  all_data = company.get_modules(modules)

  name = all_data[code]['price']['shortName']
  marketCap = all_data[code]['price']['marketCap']
  industry = all_data[code]['assetProfile']['industry']
  summary = all_data[code]['assetProfile']['longBusinessSummary']

  currentPrice = all_data[code]['financialData']['currentPrice']
  targetPrice = all_data[code]['financialData']['targetMeanPrice']

  per = all_data[code]['summaryDetail']['trailingPE']
  eps = all_data[code]['defaultKeyStatistics']['trailingEps']
  pbr = all_data[code]['defaultKeyStatistics']['priceToBook']


  doc = company.earnings[code]['financialsChart']['yearly']
  df = pd.DataFrame(doc)

  rev2019 = df.iloc[0,1] / 1000
  rev2020 = df.iloc[1,1] / 1000
  rev2021 = df.iloc[2,1] / 1000

  ear2019 = df.iloc[0,2] / 1000
  ear2020 = df.iloc[1,2] / 1000
  ear2021 = df.iloc[2,2] / 1000

  doc = {
      'code':code,
      'name':name,
      'industry':industry,
      'marketCap':marketCap/1000,
      'currentPrice':currentPrice,
      'targetPrice':targetPrice,
      'per':per,
      'eps':eps,
      'pbr':pbr,
      'rev2021':rev2021/1000,
      'rev2020':rev2020/1000,
      'rev2019':rev2019/1000,
      'ear2021':ear2021/1000,
      'ear2020':ear2020/1000,
      'ear2019':ear2019/1000,
  }

  return doc

In [ ]:
df = pd.DataFrame()

codes = ['AAPL','ABNB','BIDU','META','GOOG','MSFT','TSLA','PYPL','NFLX','NVDA']

for code in codes:
  try:
    row = add_company(code)
    df = df.append(row, ignore_index = True)
  except:
    print(f'error - {code}')

df

In [50]:
df.sort_values(by='eps',ascending = False)

df[df['per'] < 30].sort_values(by='per', ascending=False)

,code,name,industry,marketCap,currentPrice,targetPrice,per,eps,pbr,rev2021,rev2020,rev2019,ear2021,ear2020,ear2019
5,MSFT,Microsoft Corporation,Software—Infrastructure,2.080021e+09,279.43,286.22,29.506863,9.47,11.362638,168088.0,143015.0,125843.0,61271.0,44281.0,39240.0
0,AAPL,Apple Inc.,Consumer Electronics,2.465058e+09,155.00,169.23,26.315790,5.89,43.283997,365817.0,274515.0,260174.0,94680.0,57411.0,55256.0
3,META,"Meta Platforms, Inc.",Internet Content & Information,5.313100e+08,195.61,217.00,23.040049,8.49,4.067412,117929.0,85965.0,70697.0,39370.0,29146.0,18485.0
4,GOOG,Alphabet Inc.,Internet Content & Information,1.306457e+09,102.46,124.78,21.169420,4.84,5.139704,257637.0,182527.0,161857.0,76033.0,40269.0,34343.0


In [56]:
new_df = df[['name','currentPrice','targetPrice']].copy()

new_df['gap'] = new_df['targetPrice'] / new_df['currentPrice'] - 1

new_df.sort_values(by='gap', ascending=False)

,name,currentPrice,targetPrice,gap
7,"PayPal Holdings, Inc.",72.99,100.95,0.38
2,"Baidu, Inc.",146.84,180.10,0.23
4,Alphabet Inc.,102.46,124.78,0.22
8,"Netflix, Inc.",303.50,357.23,0.18
1,"Airbnb, Inc.",118.15,138.16,0.17
3,"Meta Platforms, Inc.",195.61,217.00,0.11
0,Apple Inc.,155.00,169.23,0.09
6,"Tesla, Inc.",180.13,195.99,0.09
5,Microsoft Corporation,279.43,286.22,0.02
9,NVIDIA Corporation,257.25,249.44,-0.03
